In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision.models as models
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import os
import cv2
from tqdm import tqdm
import random
from matplotlib import pyplot as plt
import pandas as pd
from collections import namedtuple
from sklearn.metrics import accuracy_score
from copy import deepcopy

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split

### SEED

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

# Dataset

In [ ]:
import cv2
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, imgs_path: list, labels_path: list=None, mode='train'):
        self.imgs_path = imgs_path
        self.labels_path = labels_path
        self.mode = mode

    def __len__(self):
        return len(self.imgs_path)

    def __getitem__(self, idx):
        img_path = self.imgs_path[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 변환

        if self.mode == 'train' or self.mode == 'valid':
            label_path = self.labels_path[idx]
            with open(label_path, 'r') as file:
                lines = file.readlines()
            
            # YOLO  label dataset의 형식은 [class_id, x_center, y_center, width, height]
            labels = [[float(x) for x in line.strip().split()] for line in lines]
            
            return img, labels

        else: # test일 때
            return img

# Train

### Data load

In [13]:
BASE = '/mnt/d/Jupyter-Goodyoung'
SAVE_PATH = f"{BASE}/save"
MODEL_SAVE = f'{SAVE_PATH}/YOLO.pth' #
WORKERS = 2
EPOCHS = 50 # 훈련 epoch 지정
BATCH_SIZE = 16# batch size 지정
IMAGE_SIZE = (256, 256) # 이미지 크기 지정
RANDOM_STATE = 42 # seed 고정
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

seed_everything(RANDOM_STATE) # SEED 고정

In [18]:
TL_files[0][:-4]

'/mnt/d/Jupyter-Goodyoung/diabetes/data/Train/TL/TL1/A/13/A13001/30/정위/A_13_A13001_가자미구이_30_09.'

In [ ]:
# 파일에서 문자열을 읽어와 리스트에 저장하는 함수
def read_strings_from_file(filename):
    strings = []
    with open(filename, 'r') as f:
        if filename == "TL_files.txt" or filename == "VL_files.txt" :
            for line in f:
                line = line.strip()
                line = line[:-4] +"txt"
                strings.append(line)  # 줄바꿈 문자 제거            
        else:    
            for line in f:
                strings.append(line.strip())  # 줄바꿈 문자 제거
    return strings
TL_files = read_strings_from_file('TL_files.txt')
TS_files = read_strings_from_file('TS_files.txt')
VL_files = read_strings_from_file('VL_files.txt')
VS_files = read_strings_from_file('VS_files.txt')

# train : val = 8 : 2 나누기
combined_imgs = TS_files + VS_files
combined_label = TL_files + VL_files
x_tr, x_val = train_test_split(combined_train, test_size=0.2, random_state=RANDOM_STATE)
y_tr, y_val = train_test_split(combined_label, test_size=0.2, random_state=RANDOM_STATE) # 둘 다 같은 비율로 나뉘어 진다

train_imgs_transform = A.Compose([
    A.OneOf([
        A.RandomRotate90(p=0.3),
        A.VerticalFlip(p=0.3),
        A.HorizontalFlip(p=0.3)
    ])
])

train_masks_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomVerticalFlip(p=0.5),
])

train_dataset = CustomDataset(x_tr, y_tr, mode = 'train')
valid_dataset = CustomDataset(x_val, y_val, mode = 'valid')

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=WORKERS,
    pin_memory=True
)
val_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=WORKERS,
    pin_memory=True
        )\

### Model train

#### model selection

In [ ]:
# torch init cache
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [3]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)

Using cache found in /home/qkboo/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-6 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12287MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=0.001)#(params=model.parameters(), lr=0.0001)
loss_fn = nn.BCEWithLogitsLoss().to(DEVICE)

In [ ]:
best_val_loss = 0 
best_model = None
early_stop = 0

model.to(DEVICE)
model.load_state_dict(torch.load(MODEL_SAVE))

for epoch in range(EPOCHS+1):
    model.train()
    print('Epoch {}/{}'.format(epoch, EPOCHS))
    print('-'*20)
    train_loss = 0

    #### train ####
    for imgs, label in tqdm(train_dataloader):
        imgs = imgs.to(DEVICE)
        label = label.to(DEVICE)
        # 초기화
        optimizer.zero_grad() 
        # 예측
        output = model(imgs) 
        # 순전파
        loss = loss_fn(output, label).to(DEVICE) 
        # 역전파
        loss.backward() 
        # 학습
        optimizer.step() 
        
        train_loss += loss.item()

    #### valid ####
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for imgs, label in tqdm(val_dataloader):
            imgs = imgs.to(DEVICE)
            label = label.to(DEVICE)
            # 예측
            output = model(imgs)
            # 순전파
            loss = loss_fn(output, label).to(DEVICE)
            
            val_loss+=loss.item()
            
    
    print(f"EPOCH: {epoch}, TRAIN LOSS: {train_loss:.6f},  VAL LOSS: {val_loss:.6f}")
    
    if lr_scheduler is not None:
        lr_scheduler.step(val_loss)
        
    if best_val_loss > val_loss:
        print("Model Save")
        
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_SAVE)
        early_stop = 0
        file_path = f'{SAVE_PATH}result_all.txt'
        with open(file_path, 'a') as file:
            file.write(f"[BEST]: EPOCH: {epoch}, TRAIN LOSS: {train_loss:.6f}, TRAIN mIou: {train_iou:.10f}, VAL LOSS: {val_loss:.6f}, VAL mIou: {val_iou:.6f}\n")
    else:
        early_stop += 1

    # early stop
    if early_stop > 22:
        print("Early Stop")
        break

# Inference

In [ ]:
y_pred_dict = {}
# data load

# load test path
test_dataset = CustomDataset(images_test, masks_test, mode = 'test')

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=1,
    shuffle=False
)

model.load_state_dict(torch.load(MODEL_SAVE))
model.to(DEVICE)
model.eval()
with torch.no_grad():
    for idx, imgs in enumerate(tqdm(test_dataloader)):
        name = test_meta['test_img'][idx]
        imgs = imgs.to(DEVICE)
        output = model(imgs)
        
        # y_pred = F.sigmoid(output)
        # pred = y_pred.cpu().detach().numpy()
        # y_pred = y_pred.astype(np.uint8)

        y_pred_dict[name] = y_pred
#
import joblib
# joblib.dump(y_pred_dict, f'{BASE}/y_pred-23.pkl')